In [3]:
import cv2
import csv

id= int(input("Enter student Registration number"))
name= input("Enter student name")

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        
        if faces == ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
            cv2.imshow("Cropped face", face)
            
        if cv2.waitKey(1)==13 or int(img_id)==50: 
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

    with open("C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/Records.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Reg no.", "Name"])  # Header
        writer.writerow([id, name]) 
generate_dataset()

Enter student Registration number20225015
Enter student nameashish


C:\Users\Shresth Jalan\AppData\Local\Temp\ipykernel_17088\1852872714.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


In [4]:
import os
import cv2
from PIL import Image 
import numpy as np

def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/classifier.xml")
train_classifier("C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/data")

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
from cv2 import face

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
        
        id, pred = clf.predict(gray_img[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
        
        if confidence>70:
            if id==20225022:
                cv2.putText(img, "Ayushman", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==20225085:
                cv2.putText(img, "Shresth", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)    
            
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
    
    return img


faceCascade = cv2.CascadeClassifier("C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/haarcascade_frontalface_default.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("C:/Users/Shresth Jalan/Downloads/p1frs/p1/backen/classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
    cv2.imshow("face Detection", img)
    
    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows()